In [1]:
from pymongo import MongoClient
import numpy as np
import pandas as pd
import json

In [12]:
client = MongoClient('localhost')
db = client.Returns_WealthNFunds

In [13]:
def insertMongo(collectionName,data):
    db.get_collection(collectionName).drop()
    db.get_collection(collectionName).insert_many(json.loads(data.reset_index().to_json(orient='records',date_format='iso')))

def readMongo(collectionName):
    data  = pd.DataFrame(list(db.get_collection(collectionName).find()))

    return data

In [18]:
caData = pd.read_csv('../Src/excel/input/priceData/CA_DATA_11.csv')
priceJanJune17 = pd.read_csv('../Src/excel/input/priceData/Price Data Jan to Jun 17.csv')
priceJulDec17 = pd.read_csv('../Src/excel/input/priceData/Price Data Jul to Dec 17.csv')

# replace missing dates of BC_TO_DATE <-- BSE_NSE_EX_DATE
caData = caData.fillna({
    "BC_TO_DATE":caData["BSE_NSE_EX_DATE"]
})
caData['BC_TO_DATE'] = pd.to_datetime(caData['BC_TO_DATE'],format='%d/%m/%Y')

In [29]:
def xformation(portfolio,price):
    price['ISIN'] = price['ISIN'].fillna(0)
    price['MTMDATE'] = pd.to_datetime(price['MTMDATE'],format='%d-%m-%Y')

    # getting all dates prices by merge and ffill/bfill
    dateDf = pd.DataFrame(index = pd.date_range(price['MTMDATE'].min(),price['MTMDATE'].max()))

    distPrice = price.drop_duplicates(subset=["ISIN","MTMDATE"]).reset_index().drop(columns='index')
    pivotDistPrice = distPrice.pivot_table(index="MTMDATE",columns="ISIN",values="MTMPRICE")

    cols = distPrice['ISIN'].unique()
    cols = cols[~pd.isna(cols)]

    pivotDf = pd.merge(dateDf,pivotDistPrice,left_index=True,right_index=True,how="left").fillna(method='ffill').fillna(method='bfill')
    pivotDf = pivotDf[cols]
    pivotDf = pivotDf.rename_axis(None, axis=1).stack().reset_index().rename(columns={'level_0':"Date",'level_1':"ISIN",0:"MTMPRICE"})
    
    distCaData = caData.drop_duplicates(subset=["ISIN","BC_TO_DATE"]).reset_index().drop(columns='index')
    print(distCaData.dtypes)
    print(pivotDf.dtypes)
    mergeDF = pd.merge(pivotDf,distCaData,left_on=["ISIN","Date"],right_on=["ISIN","BC_TO_DATE"],how='left')
    mergeDF['OLD_RATIO'] = mergeDF['OLD_RATIO'].fillna(1)
    mergeDF['NEW_RATIO'] = mergeDF['NEW_RATIO'].fillna(1)
    
    finalDf = pd.DataFrame()
    finalDf['Date'] = mergeDF['Date']
    finalDf['ISIN'] =mergeDF['ISIN']
    finalDf['close'] = mergeDF['MTMPRICE']
    finalDf['dividend'] = mergeDF['DIVIDEND_RATE'].fillna(0)
    finalDf['split_ratio'] = (mergeDF['OLD_RATIO']/mergeDF['NEW_RATIO'])
    finalDf.set_index('Date',inplace=True)
    finalDf.sort_index(inplace=True)   
    
    rawPrices = finalDf.pivot_table(index='Date',columns='ISIN',values='close')[cols]
    dividend = finalDf.pivot_table(index='Date',columns='ISIN',values='dividend')[cols]
    splitRatio = finalDf.pivot_table(index='Date',columns='ISIN',values='split_ratio')[cols]
    
    return rawPrices,dividend,splitRatio

In [19]:
# get all 2017 prices 
price = priceJanJune17.append(priceJulDec17)

In [30]:
r,d,sr = xformation(caData,price)

EVENT_TYPE                          object
BC_NUMBER                            int64
COMPANY_NAME                        object
BSE_OR_NSE_CODE                     object
ISIN                                object
BC_FROM_DATE_RECORD_DATE            object
BC_TO_DATE                  datetime64[ns]
BSE_NSE_EX_DATE                     object
REMARKS                             object
DIVIDEND_RATE                      float64
DIVIDEND_AMOUNT_SHARE              float64
A_U_C_D_FLAG                        object
NEW_RATIO                          float64
OLD_RATIO                          float64
ISSUE_PRICE                        float64
SUBSCRIPTION_START_DATE            float64
SUBSCRIPTION_END_DATE              float64
NOTES                              float64
EXCHANGE                            object
ISIN_EFF_DATE                       object
OLD_ISIN                           float64
OLD_NAME                           float64
PAYDATE                             object
FACEVALUE  

In [18]:
insertMongo("rawPrices",r)
insertMongo("dividend",d)
insertMongo("splitRatio",sr)

### Incremental Code

In [9]:
newPrice = pd.read_csv('excel/input/incremental/newDate.csv')
# insertMongo("newPrice",newPrice)

In [10]:
newPrice

,ISIN,SECURITYSYMBOL,MTMPRICE,MTMDATE,Unnamed: 4
0,CBLO 12-01-2018/15-01-2018,CBLO 12-01-2018/15-01-2018,100.0000,1/1/2018,NaN
1,INE115A07KC0,LIC HOUSING FINANCE LTD...8.02% NCD 18-02-2020,100.0568,1/1/2018,NaN
2,INE115A07DY9,LIC HOUSING FINANCE LTD...8.31% NCD 22-05-2018,100.0520,1/1/2018,NaN
3,INE115A07KH9,LIC HOUSING FINANCE LTD... 7.98% NCD 15-05-2020,99.9112,1/1/2018,NaN
4,INE115A07JV2,LIC HOUSING FINANCE LTD... 8.47% NCD 07-07-2020,100.9601,1/1/2018,NaN
5,INE115A07GB0,LIC HOUSING FINANCE LTD.(PUT-10.05.2017) 8.70%...,101.1524,1/1/2018,NaN
6,INE115A07GO3,LIC HOUSING FINANCE LTD... 8.47% NCD 21-01-2020,100.8847,1/1/2018,NaN
7,INE115A07LA2,LIC HOUSING FINANCE LTD... 7.35% NCD 16-02-2018,100.0284,1/1/2018,NaN
8,INE115A07LE4,LIC HOUSING FINANCE LTD... 7.51% NCD 14-08-2018,99.7024,1/1/2018,NaN
9,INE115A07LG9,LIC HOUSING FINANCE LTD... 7.5072% NCD 20-11-2018,99.6604,1/1/2018,NaN


In [218]:
# newPrice= readMongo("newPrice")
newPrice['MTMDATE']=newPrice['MTMDATE'].str.replace('/','-')

r1,d1,sr1 = xformation(caData,newPrice)
rawPrices = r.append(r1,sort=False).fillna(method='ffill').fillna(method='bfill')
dividend = d.append(d1,sort=False).fillna(method='ffill').fillna(method='bfill')
splitRatio = sr.append(sr1,sort=False).fillna(method='ffill').fillna(method='bfill')

In [221]:
insertMongo("rawPrices",rawPrices)
insertMongo("dividend",dividend)
insertMongo("splitRatio",splitRatio)

In [ ]:
# rawPrices.to_csv('excel/testcases/rawAppend.csv')
# dividend.to_csv('excel/testcases/dividendAppend.csv')
# splitRatio.to_csv('excel/testcases/splitRatioAppend.csv')

### NAV

In [222]:
navData = pd.read_csv('excel/input/NAV DATA.csv')

In [224]:
navData

,NAVDATE,PORTCODE,PLAN,NAV,NETASSETS
0,01-01-2017,1,AB,76.2113,1.104292e+06
1,01-01-2017,1,AD,42.9082,2.635619e+08
2,01-01-2017,1,AG,449.5133,6.606566e+08
3,01-01-2017,1,BP,73.8253,2.386099e+08
4,01-01-2017,1,DP,41.6068,6.644867e+09
5,01-01-2017,1,GP,438.3388,2.116863e+10
6,01-01-2017,1,ID,267.9983,3.146300e+02
7,01-01-2017,2,AB,142.2769,2.765993e+06
8,01-01-2017,2,AD,73.2537,5.628991e+08
9,01-01-2017,2,AG,856.3841,1.680807e+09


###  Xformation Code Break-up

In [79]:
price['ISIN'] = price['ISIN'].fillna(0)
price['MTMDATE'] = pd.to_datetime(price['MTMDATE'],format='%d-%m-%Y')

distPrice = price.drop_duplicates(subset=["ISIN","MTMDATE"]).reset_index().drop(columns='index')

# getting all dates prices by merge and ffill/bfill
dateDf = pd.DataFrame(data = {"Date": pd.date_range(price['MTMDATE'].min(),price['MTMDATE'].max())})
dateDistPrice = pd.merge(dateDf,distPrice,left_on="Date",right_on=["MTMDATE"],how="left")

In [84]:
distPrice['MTMDATE'].unique().size

365

In [86]:
dateDf.size

365

In [35]:
cols = dateDistPrice['ISIN'].unique()
cols = cols[~pd.isna(cols)]

pivotDf = dateDistPrice.pivot_table(index =['Date'],columns=['ISIN']).fillna(method='ffill').fillna(method='bfill')
pivotDf.columns=pivotDf.columns.droplevel(0)
pivotDf = pivotDf.rename_axis(None, axis=1)
pivotDf = pivotDf[cols]
pivotDf = pivotDf.rename_axis(None, axis=1).stack().reset_index().rename(columns={'level_1':"ISIN",0:"MTMPRICE"})

In [ ]:
distCaData = caData.drop_duplicates(subset=["ISIN","BC_TO_DATE"]).reset_index().drop(columns='index')
mergeDF = pd.merge(pivotDf,distCaData,left_on=["ISIN","Date"],right_on=["ISIN","BC_TO_DATE"],how='left')
mergeDF['OLD_RATIO'].fillna(1,inplace=True)
mergeDF['NEW_RATIO'].fillna(1,inplace=True)

In [37]:
finalDf = pd.DataFrame()
finalDf['Date'] = mergeDF['Date']
finalDf['ISIN'] =mergeDF['ISIN']
finalDf['close'] = mergeDF['MTMPRICE']
finalDf['dividend'] = mergeDF['DIVIDEND_RATE'].fillna(0)
finalDf['split_ratio'] = (mergeDF['OLD_RATIO']/mergeDF['NEW_RATIO']).fillna(1)
finalDf.set_index('Date',inplace=True)
finalDf.sort_index(inplace=True)

In [38]:
rawPrices = finalDf.pivot_table(index='Date',columns='ISIN',values='close')[cols]
dividend = finalDf.pivot_table(index='Date',columns='ISIN',values='dividend')[cols]
splitRatio = finalDf.pivot_table(index='Date',columns='ISIN',values='split_ratio')[cols]

In [39]:
# rawPrices.to_excel('excel/output/rawPrices.xlsx')
# dividend.to_excel('excel/output/dividend.xlsx')
# splitRatio.to_excel('excel/output/splitRatio.xlsx')

In [40]:
rawPrices.to_csv('excel/output/rawPrices.csv')
dividend.to_csv('excel/output/dividend.csv')
splitRatio.to_csv('excel/output/splitRatio.csv')

In [68]:
with pd.option_context('display.max_rows',None):
    print(mergeDF[mergeDF['NEW_RATIO']>1])

             Date          ISIN  MTMPRICE EVENT_TYPE  BC_NUMBER COMPANY_NAME  \
844541 2017-06-15  INE376G01013     344.8      BONUS   170311.0  Biocon Ltd.   

       BSE_OR_NSE_CODE BC_FROM_DATE_RECORD_DATE BC_TO_DATE BSE_NSE_EX_DATE  \
844541          532523               17/06/2017 2017-06-15      15/06/2017   

             ...       EXCHANGE  ISIN_EFF_DATE  OLD_ISIN OLD_NAME     PAYDATE  \
844541       ...            BSE     13/06/2017       NaN      NaN  26/06/2017   

        FACEVALUE  MARKETLOT  GROUPTYPE  SECURITYTYPE  SETTLEMENTMODE  
844541        NaN        NaN        NaN           NaN             NaN  

[1 rows x 30 columns]
